In [1]:
data = [i.strip('\n').split('\t') for i in open('/Users/Diego/Documents/School/ISTA539/final-project-dalcantara7/openclass_prize.train', 'r', encoding='utf8', errors='ignore')]
print(data[0])
print(" ")
print(data[1])

['What is the Turing Test?', 'The original Turing test involved three entities communicating with each other through computer terminals in separate rooms - a computer program trying to pass itself off as a human, a human trying to prove that she or he is human, and a human judge deciding which of the competitors is human and which a machine. This is different from the popular idea that it\'s about a human judging whether one subject is human or machine. The reason it\'s different is that Turing\'s original proposal was not intended as a practical test, but rather as a thought experiment making a philosophical argument - that it is impossible to determine whether something is "intelligent" better than by judging its linguistic interactions, and that therefore a machine that sounds intelligent has to be considered truly intelligent. And by "intelligent," Turing meant, like a human, with consciousness.', 'Which of the following is NOT an objection to the Turing Test mentioned in the readi

In [2]:
X = []
Y = []
for i in range(len(data)):
    if i % 2 == 0:
        X.append(' '.join(data[i]))
    else:
        Y.append(data[i])

print(X[0:3])
print(" ")
print(Y[0:5])

['What is the Turing Test? The original Turing test involved three entities communicating with each other through computer terminals in separate rooms - a computer program trying to pass itself off as a human, a human trying to prove that she or he is human, and a human judge deciding which of the competitors is human and which a machine. This is different from the popular idea that it\'s about a human judging whether one subject is human or machine. The reason it\'s different is that Turing\'s original proposal was not intended as a practical test, but rather as a thought experiment making a philosophical argument - that it is impossible to determine whether something is "intelligent" better than by judging its linguistic interactions, and that therefore a machine that sounds intelligent has to be considered truly intelligent. And by "intelligent," Turing meant, like a human, with consciousness. Which of the following is NOT an objection to the Turing Test mentioned in the reading? In

In [3]:
#may not need any of this with the glove embedding I am using
from nltk.stem.snowball import SnowballStemmer
import re

def clean_data(dataset):
    clean_data = []
    sno = SnowballStemmer('english')

    for entry in dataset:
        lowered = entry.lower()
        sans_special_chars = re.sub(r'\W',' ', lowered)
        sans_extra_spaces = re.sub(r'\s+',' ', sans_special_chars)
        sans_underscore = sans_extra_spaces.replace('_', ' ')
        split = sans_underscore.split()
        stemmed = ' '.join([sno.stem(word) for word in split])
        clean_data.append(stemmed)

    return clean_data

In [4]:
# clean_X = clean_data(X)
# print(clean_X[0:3])

In [5]:
labels = [item for sublist in Y for item in sublist]
print(labels[0:5])

['Turing test', 'Alan Turing', 'Computational linguistics', 'Language philosophy', 'Artificial Intelligence']


In [6]:
print(len(labels))

716


In [7]:
import numpy as np

sample_labels = np.zeros((len(Y), len(labels)))

for i in range(len(Y)):
    for j in range(len(Y[i])):
        for k in range(len(labels)):
            if Y[i][j] == labels[k]:
                sample_labels[i][k] = 1
                break

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, sample_labels, test_size=0.2)
print("Length of X: ", len(X))
print("Length of Y: ", len(Y))
print("Length of X_train: ", len(X_train))
print("Length of Y_train: ", len(Y_train))

Length of X:  163
Length of Y:  163
Length of X_train:  130
Length of Y_train:  130


In [9]:
from keras.preprocessing.text import Tokenizer
import keras.preprocessing.sequence as kps
NUM_WORDS = 120000

tokenizer = Tokenizer(num_words=NUM_WORDS)
tokenizer.fit_on_texts(X_train)

train_indices = tokenizer.texts_to_sequences(X_train)
word_index = tokenizer.word_index
train_indices = kps.pad_sequences(train_indices, padding='post')

test_indices = tokenizer.texts_to_sequences(X_test)
test_indices = kps.pad_sequences(test_indices, padding='post', maxlen=len(train_indices[0]))

print("Train Data: ", train_indices[0:3])
print("Test Data: ", test_indices[0:3])

Using TensorFlow backend.
Train Data:  [[346   7  19 ...   0   0   0]
 [ 55   7   4 ...   0   0   0]
 [471   1 174 ...   0   0   0]]
Test Data:  [[  85  114 8917 ...    0    0    0]
 [  55   52  683 ...    0    0    0]
 [ 803    4    7 ...    0    0    0]]


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier

clf = OneVsRestClassifier(LogisticRegression(solver='saga', class_weight={0 : 1, 1 : 150}, max_iter=50)).fit(train_indices, Y_train)

e-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 591 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 592 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 593 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 595 is present in all training examples.
  str(classes[c]))
/usr/local/lib/pytho

In [13]:
predictions_prob = clf.predict_proba(test_indices)

In [18]:
predictions = np.where(predictions_prob > 0.7, 1, 0)
predictions[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [19]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, _, __ = precision_recall_fscore_support(Y_test, predictions, average='macro')
print((5 *  precision * recall) / (4 * (precision+recall)))
precision, recall, _, __ = precision_recall_fscore_support(Y_test, predictions, average='micro')
print((5 *  precision * recall) / (4 * (precision+recall)))
precision, recall, _, __ = precision_recall_fscore_support(Y_test, predictions, average='weighted')
print((5 *  precision * recall) / (4 * (precision+recall)))

0.0009317748142408658
0.03918495297805643
0.01192634895698725
